In [1]:
import argparse
import numpy as np
import os
import sys
import yaml
import torch
import torchmetrics

from argparse import Namespace
from lib import utils
from lib.utils import load_graph_data
from model.pytorch.dcrnn_supervisor import DCRNNSupervisor

In [2]:
def get_supervisor(config_filename, model_path):
    args = Namespace(
        use_cpu_only = False,
        config_filename=config_filename
    )
    
    with open(args.config_filename) as f:
        supervisor_config = yaml.load(f)
        
        graph_pkl_filename = supervisor_config['data'].get('graph_pkl_filename')
        sensor_ids, sensor_id_to_ind, adj_mx = load_graph_data(graph_pkl_filename)
        
        supervisor = DCRNNSupervisor(adj_mx = adj_mx, **supervisor_config)
        supervisor._setup_graph()
        
        checkpoint = torch.load(model_path, map_location='cpu')
        supervisor.dcrnn_model.load_state_dict(checkpoint['model_state_dict'])
        
        return supervisor

In [3]:
def get_prediction_data(supervisor):
    with torch.no_grad():
        supervisor.dcrnn_model = supervisor.dcrnn_model.eval()
        
        val_iterator = supervisor._data['test_loader'].get_iterator()
        y_preds = []
        
        for _, (x, y) in enumerate(val_iterator):
            x, y = supervisor._prepare_data(x, y)
            output = supervisor.dcrnn_model(x)
            y_preds.append(output.cpu())
            
        y_preds = np.concatenate(y_preds, axis=1)
        y_preds_scaled = []
        
        for t in range(y_preds.shape[0]):
            y_pred = supervisor.standard_scaler.inverse_transform(y_preds[t])
            y_preds_scaled.append(y_pred)
            
        return y_preds_scaled

In [4]:
def get_truth_data(supervisor, dataset_dir):
    supervisor._data = utils.load_dataset(dataset_dir = dataset_dir, 
                                          batch_size = 64, 
                                          val_batch_size = 64,
                                          test_batch_size=64)
    supervisor.standard_scaler = supervisor._data['scaler']
    val_iterator = supervisor._data['test_loader'].get_iterator()
    y_truths = []
    
    for _, (x, y) in enumerate(val_iterator):
        x, y = supervisor._prepare_data(x, y)
        y_truths.append(y.cpu())
    y_truths = np.concatenate(y_truths, axis=1)
    y_truth_scaled = []
    
    for t in range(y_truths.shape[0]):
        y_truth = supervisor.standard_scaler.inverse_transform(y_truths[t])
        y_truth_scaled.append(y_truth)
        
    return y_truth_scaled

In [5]:
def evaluation(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)

In [77]:
supervisor = get_supervisor('data/model/dcrnn_virtual_0.yaml', 'models/virtual_0/epo32.tar')
y_pred = get_prediction_data(supervisor)
y_true = get_truth_data(supervisor, 'data/virtual_0')
print('Virtual No Drop 0%')

2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001402/
2023-05-18 00:14:02,525 - INFO - Log 

C:\Users\vfgtr554\AppData\Local\Temp\ipykernel_24928\1902753131.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  supervisor_config = yaml.load(f)


Virtual No Drop 0%


In [78]:
def evaluation(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=df.columns)
        predDf = pd.DataFrame(prediction, columns=df.columns)
        for col in predDf.columns:
            if '_' in col:
                predDf = predDf.drop(columns=[col])
                
        for col in trueDf.columns:
            if '_' in col:
                trueDf = trueDf.drop(columns=[col])
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)

In [39]:
evaluation(y_pred, y_true)

2
15 Min
MAE: tensor(2.3264) RMSE: tensor(3.3644) MAPE: 10.009554874749831 MAPE2: tensor(10.0096)
5
30 Min
MAE: tensor(2.6477) RMSE: tensor(3.7003) MAPE: 11.395621594087576 MAPE2: tensor(11.3956)
8
45 Min
MAE: tensor(2.8958) RMSE: tensor(3.9762) MAPE: 12.406031290690105 MAPE2: tensor(12.4060)
11
60 Min
MAE: tensor(3.0703) RMSE: tensor(4.1554) MAPE: 13.104511779031636 MAPE2: tensor(13.1045)


In [79]:
def evaluation_only(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=df.columns)
        predDf = pd.DataFrame(prediction, columns=df.columns)
        for col in predDf.columns:
            if '_' in col:
                predDf = predDf.drop(columns=[col])
                
        for col in trueDf.columns:
            if '_' in col:
                trueDf = trueDf.drop(columns=[col])
        
        columns = ['1220025300', '1220029900', '1220026000', '1220024700',
                  '1220022300', '1220026100', '1220029000']
        
        predDf = predDf[columns]
        trueDf = trueDf[columns]
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)

In [80]:
evaluation_only(y_pred, y_true)

2
15 Min
MAE: tensor(2.2174) RMSE: tensor(3.3400) MAPE: 9.12875902085077 MAPE2: tensor(9.1288)
5
30 Min
MAE: tensor(2.5852) RMSE: tensor(3.7255) MAPE: 10.562787737165179 MAPE2: tensor(10.5628)
8
45 Min
MAE: tensor(2.8745) RMSE: tensor(4.0736) MAPE: 11.684100590054951 MAPE2: tensor(11.6841)
11
60 Min
MAE: tensor(3.0775) RMSE: tensor(4.2371) MAPE: 12.503272011166528 MAPE2: tensor(12.5033)


In [59]:
supervisor = get_supervisor('data/model/dcrnn_virtual_5.yaml', 'models/virtual_5/epo21.tar')
y_pred = get_prediction_data(supervisor)
y_true = get_truth_data(supervisor, 'data/virtual_0')
print('virtual 5%')

2023-05-18 00:08:51,494 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518000851/
2023-05-18 00:08:51,494 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518000851/
2023-05-18 00:08:51,494 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518000851/
2023-05-18 00:08:51,494 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518000851/
2023-05-18 00:08:51,494 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518000851/
2023-05-18 00:08:51,494 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518000851/
2023-05-18 00:08:51,494 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518000851/
2023-05-18 00:08:51,638 - INFO - Model created
2023-05-18 00:08:51,638 - INFO - Model created
2023-05-18 00:08:51,638 - INFO - Model created
2023-05-18 00:08:51,638 - INFO - Model created
2023-05-18 00:08:51,638 - INFO - Model created
2023-05-18 00:08

C:\Users\vfgtr554\AppData\Local\Temp\ipykernel_24928\1902753131.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  supervisor_config = yaml.load(f)


virtual 5%


In [ ]:
#Mean

In [45]:
def evaluation_5(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=df.columns)
        predDf = pd.DataFrame(prediction, columns=df.columns)
        
        predDf['1220025300'] = (predDf['1220026300_1'] + predDf['1220025300_3'])/2
        predDf['1220029900'] = (predDf['1220027600_2'] + predDf['1220031100_3'])/2
        
        for col in predDf.columns:
            if '_' in col:
                predDf = predDf.drop(columns=[col])
                
        for col in trueDf.columns:
            if '_' in col:
                trueDf = trueDf.drop(columns=[col])
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)

In [46]:
evaluation_5(y_pred, y_true)

2
15 Min
MAE: tensor(2.4446) RMSE: tensor(3.4947) MAPE: 10.56440023728359 MAPE2: tensor(10.5644)
5
30 Min
MAE: tensor(2.7422) RMSE: tensor(3.7961) MAPE: 11.895788451771677 MAPE2: tensor(11.8958)
8
45 Min
MAE: tensor(2.9906) RMSE: tensor(4.0691) MAPE: 12.921436922049818 MAPE2: tensor(12.9214)
11
60 Min
MAE: tensor(3.1986) RMSE: tensor(4.2780) MAPE: 13.801471097969713 MAPE2: tensor(13.8015)


In [60]:
def evaluation_5_only(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=df.columns)
        predDf = pd.DataFrame(prediction, columns=df.columns)
        
        predDf['1220025300'] = (predDf['1220026300_1'] + predDf['1220025300_3'])/2
        predDf['1220029900'] = (predDf['1220027600_2'] + predDf['1220031100_3'])/2
        
        for col in predDf.columns:
            if '_' in col:
                predDf = predDf.drop(columns=[col])
                
        for col in trueDf.columns:
            if '_' in col:
                trueDf = trueDf.drop(columns=[col])
                
        #columns = ['1220025300', '1220029900']
        columns = ['1220025300', '1220029900', '1220026000', '1220024700',
                  '1220022300', '1220026100', '1220029000']
        
        predDf = predDf[columns]
        trueDf = trueDf[columns]
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)

In [55]:
evaluation_5_only(y_pred, y_true)

2
15 Min
MAE: tensor(4.3293) RMSE: tensor(4.9548) MAPE: 18.416272269354927 MAPE2: tensor(18.4163)
5
30 Min
MAE: tensor(4.2932) RMSE: tensor(4.9602) MAPE: 18.28786399629381 MAPE2: tensor(18.2879)
8
45 Min
MAE: tensor(4.2527) RMSE: tensor(4.9870) MAPE: 17.997372150421143 MAPE2: tensor(17.9974)
11
60 Min
MAE: tensor(4.2414) RMSE: tensor(4.9947) MAPE: 17.908898989359535 MAPE2: tensor(17.9089)


In [61]:
evaluation_5_only(y_pred, y_true)

2
15 Min
MAE: tensor(2.9801) RMSE: tensor(4.0886) MAPE: 12.459476410396515 MAPE2: tensor(12.4595)
5
30 Min
MAE: tensor(3.1888) RMSE: tensor(4.2947) MAPE: 13.325131128704736 MAPE2: tensor(13.3251)
8
45 Min
MAE: tensor(3.3881) RMSE: tensor(4.5109) MAPE: 14.094728136819507 MAPE2: tensor(14.0947)
11
60 Min
MAE: tensor(3.5673) RMSE: tensor(4.6780) MAPE: 14.837598043774802 MAPE2: tensor(14.8376)


In [137]:
supervisor = get_supervisor('data/model/dcrnn_virtual_20.yaml', 'models/virtual_20/epo41.tar')
y_pred = get_prediction_data(supervisor)
y_true = get_truth_data(supervisor, 'data/virtual_0')
print('virtual 20%')

2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150244/
2023-05-18 15:02:44,721 - INFO - Log 

C:\Users\vfgtr554\AppData\Local\Temp\ipykernel_24928\1902753131.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  supervisor_config = yaml.load(f)


virtual 20%


In [82]:
def evaluation_20(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=df.columns)
        predDf = pd.DataFrame(prediction, columns=df.columns)
        
        predDf['1220025300'] = (predDf['1220026300_1'] + predDf['1220025300_3'])/2
        predDf['1220029900'] = (predDf['1220027600_2'] + predDf['1220031100_3'])/2
        predDf['1220026000'] = (predDf['1220024600_0'] + predDf['1220025200_2'])/2
        predDf['1220024700'] = (predDf['1220025800_0'] + predDf['1220027700_1'])/2
        predDf['1220022300'] = (predDf['1220023000_1'] + predDf['1220020600_3'])/2
        predDf['1220026100'] = (predDf['1220029100_1'] + predDf['1220029900_3'])/2
        predDf['1220029000'] = (predDf['1220023100_0'] + predDf['1220025500_2'])/2
        
        for col in predDf.columns:
            if '_' in col:
                predDf = predDf.drop(columns=[col])
                
        for col in trueDf.columns:
            if '_' in col:
                trueDf = trueDf.drop(columns=[col])
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)

In [83]:
evaluation_20(y_pred, y_true)

2
15 Min
MAE: tensor(3.4449) RMSE: tensor(5.2370) MAPE: 14.389981163872612 MAPE2: tensor(14.3900)
5
30 Min
MAE: tensor(3.6780) RMSE: tensor(5.4157) MAPE: 15.452487380416304 MAPE2: tensor(15.4525)
8
45 Min
MAE: tensor(3.8696) RMSE: tensor(5.5910) MAPE: 16.20101104547948 MAPE2: tensor(16.2010)
11
60 Min
MAE: tensor(4.0352) RMSE: tensor(5.7405) MAPE: 16.859332426094713 MAPE2: tensor(16.8593)


In [124]:
def evaluation_20_only(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=df.columns)
        predDf = pd.DataFrame(prediction, columns=df.columns)
        
        predDf['1220025300'] = (predDf['1220026300_1'] + predDf['1220025300_3'])/2
        predDf['1220029900'] = (predDf['1220027600_2'] + predDf['1220031100_3'])/2
        predDf['1220026000'] = (predDf['1220024600_0'] + predDf['1220025200_2'])/2
        predDf['1220024700'] = (predDf['1220025800_0'] + predDf['1220027700_1'])/2
        predDf['1220022300'] = (predDf['1220023000_1'] + predDf['1220020600_3'])/2
        predDf['1220026100'] = (predDf['1220029100_1'] + predDf['1220029900_3'])/2
        predDf['1220029000'] = (predDf['1220023100_0'] + predDf['1220025500_2'])/2
        
        for col in predDf.columns:
            if '_' in col:
                predDf = predDf.drop(columns=[col])
                
        for col in trueDf.columns:
            if '_' in col:
                trueDf = trueDf.drop(columns=[col])
                
        columns = ['1220025300', '1220029900', '1220026000', '1220024700',
                  '1220022300', '1220026100', '1220029000']
        
        predDf = predDf[columns]
        trueDf = trueDf[columns]
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)
        
    return predDf, trueDf

In [125]:
p, t = evaluation_20_only(y_pred, y_true)

2
15 Min
MAE: tensor(7.9500) RMSE: tensor(9.7009) MAPE: 30.78485519166977 MAPE2: tensor(30.7849)
5
30 Min
MAE: tensor(7.9918) RMSE: tensor(9.7738) MAPE: 30.85477617051866 MAPE2: tensor(30.8548)
8
45 Min
MAE: tensor(8.0881) RMSE: tensor(9.8931) MAPE: 31.170954023088726 MAPE2: tensor(31.1710)
11
60 Min
MAE: tensor(8.2111) RMSE: tensor(10.0082) MAPE: 31.651148720393106 MAPE2: tensor(31.6511)


In [126]:
p

,1220025300,1220029900,1220026000,1220024700,1220022300,1220026100,1220029000
0,27.073700,17.736574,20.239536,12.598211,22.822577,11.350378,25.002712
1,28.120590,17.660004,20.390446,13.034716,23.755356,11.350268,25.055901
2,28.541222,18.003069,20.911905,13.456710,23.933758,11.214125,26.981670
3,28.748446,18.191887,21.521891,14.099430,23.934727,11.189169,27.749485
4,28.952810,18.116421,21.126272,14.026608,23.593887,11.315218,28.429506
...,...,...,...,...,...,...,...
571,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
572,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
573,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
574,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628


In [127]:
t

,1220025300,1220029900,1220026000,1220024700,1220022300,1220026100,1220029000
0,22.420000,29.280001,27.000000,21.830000,26.549999,25.309999,25.100000
1,23.820000,30.440001,25.000000,23.100000,26.490000,26.020000,24.290001
2,26.620001,30.180000,24.610001,24.219999,24.139999,22.580000,23.980000
3,26.200001,30.180000,28.860001,24.629999,32.090000,25.209999,23.790001
4,27.480000,29.590000,29.040001,22.360001,30.309999,27.540001,23.870001
...,...,...,...,...,...,...,...
571,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
572,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
573,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
574,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999


In [ ]:
#Compare Model (node = 36)

In [64]:
def evaluation(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)

In [72]:
def evaluation_only(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=dfs.columns)
        predDf = pd.DataFrame(prediction, columns=dfs.columns)
                
        columns = ['1220025300', '1220029900', '1220026000', '1220024700',
                  '1220022300', '1220026100', '1220029000']
        
        predDf = predDf[columns]
        trueDf = trueDf[columns]
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)

In [73]:
supervisor = get_supervisor('data/model/dcrnn_compare_0.yaml', 'models/compare_0/epo12.tar')
y_pred = get_prediction_data(supervisor)
y_true = get_truth_data(supervisor, 'data/compare_0')
print('Compare 0%')
evaluation(y_pred, y_true)
evaluation_only(y_pred, y_true)

2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001234/
2023-05-18 00:12:34,159 - INFO - Log 

C:\Users\vfgtr554\AppData\Local\Temp\ipykernel_24928\1902753131.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  supervisor_config = yaml.load(f)


Compare 0%
2
15 Min
MAE: tensor(2.2599) RMSE: tensor(3.2967) MAPE: 9.859050939112533 MAPE2: tensor(9.8591)
5
30 Min
MAE: tensor(2.5344) RMSE: tensor(3.5738) MAPE: 11.130299980257764 MAPE2: tensor(11.1303)
8
45 Min
MAE: tensor(2.7302) RMSE: tensor(3.7903) MAPE: 11.915885077582464 MAPE2: tensor(11.9159)
11
60 Min
MAE: tensor(2.8944) RMSE: tensor(3.9589) MAPE: 12.54831714394652 MAPE2: tensor(12.5483)
2
15 Min
MAE: tensor(2.1417) RMSE: tensor(3.2449) MAPE: 8.922648051428418 MAPE2: tensor(8.9226)
5
30 Min
MAE: tensor(2.4102) RMSE: tensor(3.5148) MAPE: 10.033783836970253 MAPE2: tensor(10.0338)
8
45 Min
MAE: tensor(2.6214) RMSE: tensor(3.7297) MAPE: 10.855138869512649 MAPE2: tensor(10.8551)
11
60 Min
MAE: tensor(2.7868) RMSE: tensor(3.9104) MAPE: 11.49501951913985 MAPE2: tensor(11.4950)


In [74]:
supervisor = get_supervisor('data/model/dcrnn_compare_5.yaml', 'models/compare_5/epo17.tar')
y_pred = get_prediction_data(supervisor)
y_true = get_truth_data(supervisor, 'data/compare_0')
print('Compare 5%')
evaluation(y_pred, y_true)
evaluation_only(y_pred, y_true)

2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001246/
2023-05-18 00:12:46,028 - INFO - Log 

C:\Users\vfgtr554\AppData\Local\Temp\ipykernel_24928\1902753131.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  supervisor_config = yaml.load(f)


Compare 5%
2
15 Min
MAE: tensor(3.6246) RMSE: tensor(6.9870) MAPE: 15.197636168680074 MAPE2: tensor(15.1976)
5
30 Min
MAE: tensor(3.8812) RMSE: tensor(7.1191) MAPE: 16.330318686402876 MAPE2: tensor(16.3303)
8
45 Min
MAE: tensor(4.1055) RMSE: tensor(7.2612) MAPE: 17.174446435622226 MAPE2: tensor(17.1744)
11
60 Min
MAE: tensor(4.2985) RMSE: tensor(7.3730) MAPE: 17.92229546440972 MAPE2: tensor(17.9223)
2
15 Min
MAE: tensor(9.0679) RMSE: tensor(14.3401) MAPE: 35.754654899476066 MAPE2: tensor(35.7547)
5
30 Min
MAE: tensor(9.2485) RMSE: tensor(14.3881) MAPE: 36.4489025539822 MAPE2: tensor(36.4489)
8
45 Min
MAE: tensor(9.4497) RMSE: tensor(14.4757) MAPE: 37.169898502410405 MAPE2: tensor(37.1699)
11
60 Min
MAE: tensor(9.6185) RMSE: tensor(14.5238) MAPE: 37.833580138191344 MAPE2: tensor(37.8336)


In [75]:
supervisor = get_supervisor('data/model/dcrnn_compare_10.yaml', 'models/compare_10/epo24.tar')
y_pred = get_prediction_data(supervisor)
y_true = get_truth_data(supervisor, 'data/compare_0')
print('Compare 10%')
evaluation(y_pred, y_true)
evaluation_only(y_pred, y_true)

2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518001252/
2023-05-18 00:12:52,272 - INFO - Log 

C:\Users\vfgtr554\AppData\Local\Temp\ipykernel_24928\1902753131.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  supervisor_config = yaml.load(f)


Compare 10%
2
15 Min
MAE: tensor(4.8174) RMSE: tensor(9.0734) MAPE: 19.99789814890167 MAPE2: tensor(19.9979)
5
30 Min
MAE: tensor(5.0557) RMSE: tensor(9.1649) MAPE: 21.079296535915798 MAPE2: tensor(21.0793)
8
45 Min
MAE: tensor(5.2304) RMSE: tensor(9.2510) MAPE: 21.78937652964651 MAPE2: tensor(21.7894)
11
60 Min
MAE: tensor(5.4017) RMSE: tensor(9.3332) MAPE: 22.463548919300973 MAPE2: tensor(22.4635)
2
15 Min
MAE: tensor(15.2827) RMSE: tensor(19.4550) MAPE: 61.00083608475943 MAPE2: tensor(61.0008)
5
30 Min
MAE: tensor(15.3959) RMSE: tensor(19.4821) MAPE: 61.433023119729654 MAPE2: tensor(61.4330)
8
45 Min
MAE: tensor(15.4953) RMSE: tensor(19.5223) MAPE: 61.77606734018477 MAPE2: tensor(61.7761)
11
60 Min
MAE: tensor(15.5975) RMSE: tensor(19.5559) MAPE: 62.121993776351694 MAPE2: tensor(62.1220)


In [130]:
supervisor = get_supervisor('data/model/dcrnn_compare_20.yaml', 'models/compare_20/epo18.tar')
y_pred = get_prediction_data(supervisor)
y_true = get_truth_data(supervisor, 'data/compare_0')
print('Compare 20%')
evaluation(y_pred, y_true)
evaluation_only(y_pred, y_true)

2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log directory: data/model\dcrnn_DR_2_h_12_64-64_lr_0.01_bs_64_0518150204/
2023-05-18 15:02:04,403 - INFO - Log 

C:\Users\vfgtr554\AppData\Local\Temp\ipykernel_24928\1902753131.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  supervisor_config = yaml.load(f)


Compare 20%
2


ValueError: Shape of passed values is (576, 36), indices imply (576, 72)

In [95]:
def evaluation_only(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=dfs.columns)
        predDf = pd.DataFrame(prediction, columns=dfs.columns)
                
        columns = ['1220025300', '1220029900', '1220026000', '1220024700',
                  '1220022300', '1220026100', '1220029000']
        
        predDf = predDf[columns]
        trueDf = trueDf[columns]
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)
    return predDf, trueDf

In [100]:
trueDf = pd.DataFrame(y_true[-1], columns = dfs.columns)
predDf = pd.DataFrame(y_pred[-1], columns = dfs.columns)

In [112]:
import random
columns = ['1220025300', '1220029900', '1220026000', '1220024700',
                  '1220022300', '1220026100', '1220029000']
n1, n2 = random.sample(list(trueDf.columns.drop(columns)), 2)
n1, n2

('1220025900', '1220027500')

Index(['1220026200', '1220022900', '1220019800', '1220023000', '1220026300',
       '1220029700', '1220028200', '1220027200', '1220028300', '1220029100',
       '1220030100', '1220027600', '1220025800', '1220023100', '1220025900',
       '1220027700', '1220031100', '1220027400', '1220024600', '1220021400',
       '1220027500', '1220023900', '1220025500', '1220023800', '1220022200',
       '1220020600', '1220025200', '1220024000', '1220026900'],
      dtype='object')

In [96]:
pp, tt = evaluation_only(y_pred, y_true)

2
15 Min
MAE: tensor(18.6690) RMSE: tensor(19.4361) MAPE: 74.35530405195932 MAPE2: tensor(74.3553)
5
30 Min
MAE: tensor(16.4458) RMSE: tensor(17.3167) MAPE: 64.93911743164062 MAPE2: tensor(64.9391)
8
45 Min
MAE: tensor(14.7985) RMSE: tensor(15.7969) MAPE: 57.895890493241566 MAPE2: tensor(57.8959)
11
60 Min
MAE: tensor(13.6639) RMSE: tensor(14.8346) MAPE: 53.08179098462301 MAPE2: tensor(53.0818)


In [97]:
pp

,1220025300,1220029900,1220026000,1220024700,1220022300,1220026100,1220029000
0,12.510891,17.455742,18.820700,17.455860,19.245670,12.510889,14.634655
1,12.534997,17.295586,18.413599,17.295712,19.788523,12.535000,14.440311
2,12.832245,17.214952,17.379723,17.215160,18.539108,12.832245,14.803255
3,13.734566,17.174292,19.827579,17.174507,18.635704,13.734566,14.937490
4,14.950094,18.246925,20.750587,18.247002,18.168701,14.950092,14.935769
...,...,...,...,...,...,...,...
571,14.611403,19.123451,16.909561,19.123817,14.518284,14.611403,16.215542
572,14.611403,19.123451,16.909561,19.123817,14.518284,14.611403,16.215542
573,14.611403,19.123451,16.909561,19.123817,14.518284,14.611403,16.215542
574,14.611403,19.123451,16.909561,19.123817,14.518284,14.611403,16.215542


In [99]:
p

,1220025300,1220029900,1220026000,1220024700,1220022300,1220026100,1220029000
0,27.073700,17.736574,20.239536,12.598211,22.822577,11.350378,25.002712
1,28.120590,17.660004,20.390446,13.034716,23.755356,11.350268,25.055901
2,28.541222,18.003069,20.911905,13.456710,23.933758,11.214125,26.981670
3,28.748446,18.191887,21.521891,14.099430,23.934727,11.189169,27.749485
4,28.952810,18.116421,21.126272,14.026608,23.593887,11.315218,28.429506
...,...,...,...,...,...,...,...
571,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
572,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
573,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
574,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628


In [98]:
tt

,1220025300,1220029900,1220026000,1220024700,1220022300,1220026100,1220029000
0,22.420000,29.280001,27.000000,21.830000,26.549999,25.309999,25.100000
1,23.820000,30.440001,25.000000,23.100000,26.490000,26.020000,24.290001
2,26.620001,30.180000,24.610001,24.219999,24.139999,22.580000,23.980000
3,26.200001,30.180000,28.860001,24.629999,32.090000,25.209999,23.789999
4,27.480000,29.590000,29.040001,22.360001,30.310001,27.540001,23.869999
...,...,...,...,...,...,...,...
571,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
572,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
573,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
574,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999


In [118]:
predDf = pd.DataFrame(y_pred[0], columns=dfs.columns)
predDf

,1220026200,1220022900,1220019800,1220023000,1220026300,1220029700,1220029000,1220028200,1220027200,1220028300,...,1220025500,1220023800,1220022200,1220020600,1220026000,1220025200,1220024000,1220025300,1220026100,1220026900
0,27.653597,28.161579,18.737406,25.098274,20.448620,23.281557,4.981919,13.147840,25.332209,19.039072,...,20.211617,26.268616,25.152390,15.384823,5.941598,27.288717,23.817511,3.341692,3.341690,20.428148
1,28.002716,29.236065,19.632973,24.825603,20.136564,22.980209,4.698337,12.089268,25.888386,19.065977,...,20.889439,26.816833,25.534367,14.842655,5.494384,27.386662,24.742445,3.694473,3.694473,20.139450
2,28.172541,29.532829,19.499994,25.230970,19.760000,23.596313,4.523278,11.363686,25.820309,19.619949,...,20.868368,26.491373,24.961664,14.525723,5.081299,27.390968,26.485689,4.053175,4.053175,20.172245
3,28.057955,29.791553,19.767445,25.107962,19.926708,24.266006,4.575310,10.905145,25.477062,19.558752,...,21.123756,26.644855,24.977938,14.932846,5.527630,28.902386,27.530729,4.412487,4.412489,20.159857
4,27.601971,30.231062,19.844471,24.798161,20.070091,24.505693,4.598450,11.036388,24.715805,19.884912,...,21.930729,26.765850,25.856119,14.713204,5.911316,28.815350,26.894360,4.146027,4.146027,19.572063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,30.906353,30.702896,27.859299,26.143305,22.512827,26.127783,4.597311,11.016466,26.393284,22.854095,...,23.635996,25.385302,29.032162,20.132504,7.947864,29.588108,27.264336,4.423298,4.423298,20.174820
572,30.906353,30.702896,27.859299,26.143305,22.512827,26.127783,4.597311,11.016466,26.393284,22.854095,...,23.635996,25.385302,29.032162,20.132504,7.947864,29.588108,27.264336,4.423298,4.423298,20.174820
573,30.906353,30.702896,27.859299,26.143305,22.512827,26.127783,4.597311,11.016466,26.393284,22.854095,...,23.635996,25.385302,29.032162,20.132504,7.947864,29.588108,27.264336,4.423298,4.423298,20.174820
574,30.906353,30.702896,27.859299,26.143305,22.512827,26.127783,4.597311,11.016466,26.393284,22.854095,...,23.635996,25.385302,29.032162,20.132504,7.947864,29.588108,27.264336,4.423298,4.423298,20.174820


In [131]:
import random

def evaluation_only(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=dfs.columns)
        predDf = pd.DataFrame(prediction, columns=dfs.columns)
                
        columns = ['1220025300', '1220029900', '1220026000', '1220024700',
                  '1220022300', '1220026100', '1220029000']
        for col in columns:
            n1, n2 = random.sample(list(predDf.columns.drop(columns)), 2)
            predDf[col] = (predDf[n1] + predDf[n2])/2
        
        predDf = predDf[columns]
        trueDf = trueDf[columns]
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)
    return predDf, trueDf

In [132]:
pp, tt = evaluation_only(y_pred, y_true)

2
15 Min
MAE: tensor(3.9730) RMSE: tensor(5.0465) MAPE: 16.695279923696365 MAPE2: tensor(16.6953)
5
30 Min
MAE: tensor(4.2977) RMSE: tensor(5.3431) MAPE: 17.939619034055678 MAPE2: tensor(17.9396)
8
45 Min
MAE: tensor(4.5378) RMSE: tensor(5.6066) MAPE: 18.830462864467076 MAPE2: tensor(18.8305)
11
60 Min
MAE: tensor(3.8421) RMSE: tensor(5.0439) MAPE: 16.61185612754216 MAPE2: tensor(16.6119)


In [133]:
p

,1220025300,1220029900,1220026000,1220024700,1220022300,1220026100,1220029000
0,27.073700,17.736574,20.239536,12.598211,22.822577,11.350378,25.002712
1,28.120590,17.660004,20.390446,13.034716,23.755356,11.350268,25.055901
2,28.541222,18.003069,20.911905,13.456710,23.933758,11.214125,26.981670
3,28.748446,18.191887,21.521891,14.099430,23.934727,11.189169,27.749485
4,28.952810,18.116421,21.126272,14.026608,23.593887,11.315218,28.429506
...,...,...,...,...,...,...,...
571,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
572,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
573,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628
574,28.159498,16.935188,19.719246,14.512230,25.924204,12.716492,26.669628


In [134]:
pp

,1220025300,1220029900,1220026000,1220024700,1220022300,1220026100,1220029000
0,25.548565,26.768225,27.225050,25.244442,28.795357,27.688520,26.172672
1,25.496468,26.729225,27.237789,25.997063,28.509630,27.474331,26.084835
2,26.431774,26.732550,27.439272,25.978619,28.111723,27.495724,25.753529
3,26.415051,26.475677,27.834461,25.613010,27.948784,27.359507,26.057495
4,27.514370,26.641199,28.033421,25.866940,27.566402,27.480789,27.052351
...,...,...,...,...,...,...,...
571,27.943512,29.309238,25.650873,27.050266,26.879501,30.462429,24.070890
572,27.943512,29.309238,25.650873,27.050266,26.879501,30.462429,24.070890
573,27.943512,29.309238,25.650873,27.050266,26.879501,30.462429,24.070890
574,27.943512,29.309238,25.650873,27.050266,26.879501,30.462429,24.070890


In [135]:
t

,1220025300,1220029900,1220026000,1220024700,1220022300,1220026100,1220029000
0,22.420000,29.280001,27.000000,21.830000,26.549999,25.309999,25.100000
1,23.820000,30.440001,25.000000,23.100000,26.490000,26.020000,24.290001
2,26.620001,30.180000,24.610001,24.219999,24.139999,22.580000,23.980000
3,26.200001,30.180000,28.860001,24.629999,32.090000,25.209999,23.790001
4,27.480000,29.590000,29.040001,22.360001,30.309999,27.540001,23.870001
...,...,...,...,...,...,...,...
571,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
572,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
573,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999
574,22.760000,35.160000,27.690001,24.639999,30.049999,27.719999,26.959999


In [138]:
predDf = pd.DataFrame(y_pred[-1], columns=df.columns)
predDf

,1220026200,1220022900,1220019800,1220023000,1220026300,1220029700,1220029000,1220028200,1220027200,1220028300,...,1220025500_2,1220023800_3,1220024600_0,1220027500_1,1220025200_2,1220026100_3,1220021400_0,1220024700_1,1220023900_2,1220022200_3
0,27.605373,28.654398,19.802216,25.580561,21.705139,23.653015,8.827627,10.977421,25.471380,20.316191,...,24.870363,23.875105,21.302790,17.326237,19.176285,14.626329,17.594563,17.123966,25.647118,25.908651
1,27.359293,28.674553,20.943321,24.896019,21.703604,23.613033,10.446012,11.945906,26.948906,19.939096,...,24.985008,24.047754,21.402864,17.482183,19.378029,14.655661,17.843939,16.820190,26.019524,26.161503
2,26.549520,27.570820,19.907970,24.920568,22.280649,24.127422,12.279997,12.924173,28.118534,20.387445,...,27.276049,25.907242,21.770063,17.716593,20.053747,14.856844,17.484810,16.565426,26.172604,26.343632
3,26.245512,27.185923,20.250008,24.606812,22.340012,24.265059,13.233625,13.799799,26.631111,19.878901,...,28.263458,26.497381,21.607641,17.781082,21.436140,15.624481,17.889906,16.781849,26.218969,26.352623
4,26.171389,27.821451,20.481262,24.539383,22.400585,24.398788,13.716053,14.110836,23.744764,21.312788,...,29.191906,27.330458,20.607767,16.715788,21.644777,15.438059,19.164412,17.911057,26.553833,26.591345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,28.688374,29.732599,26.330053,26.133223,26.115631,26.281403,14.504349,14.606216,29.062454,21.996822,...,27.371822,26.192446,19.926394,15.445785,19.512098,14.751400,19.852522,19.470242,25.676929,26.188145
572,28.688374,29.732599,26.330053,26.133223,26.115631,26.281403,14.504349,14.606216,29.062454,21.996822,...,27.371822,26.192446,19.926394,15.445785,19.512098,14.751400,19.852522,19.470242,25.676929,26.188145
573,28.688374,29.732599,26.330053,26.133223,26.115631,26.281403,14.504349,14.606216,29.062454,21.996822,...,27.371822,26.192446,19.926394,15.445785,19.512098,14.751400,19.852522,19.470242,25.676929,26.188145
574,28.688374,29.732599,26.330053,26.133223,26.115631,26.281403,14.504349,14.606216,29.062454,21.996822,...,27.371822,26.192446,19.926394,15.445785,19.512098,14.751400,19.852522,19.470242,25.676929,26.188145


In [141]:
t['1220025300']

0      22.420000
1      23.820000
2      26.620001
3      26.200001
4      27.480000
         ...    
571    22.760000
572    22.760000
573    22.760000
574    22.760000
575    22.760000
Name: 1220025300, Length: 576, dtype: float32

In [142]:
pp['1220025300']

0      25.548565
1      25.496468
2      26.431774
3      26.415051
4      27.514370
         ...    
571    27.943512
572    27.943512
573    27.943512
574    27.943512
575    27.943512
Name: 1220025300, Length: 576, dtype: float32

In [140]:
p['1220025300']

0      27.073700
1      28.120590
2      28.541222
3      28.748446
4      28.952810
         ...    
571    28.159498
572    28.159498
573    28.159498
574    28.159498
575    28.159498
Name: 1220025300, Length: 576, dtype: float32

In [150]:
def eval(prediction, real):
    mae = torchmetrics.functional.mean_absolute_error(
        torch.from_numpy(prediction), 
        torch.from_numpy(real)
    )
    rmse = torch.sqrt(
        torchmetrics.functional.mean_squared_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
    )

    mapes = torchmetrics.MeanAbsolutePercentageError()
    mape2 = mapes(preds = torch.from_numpy(prediction), 
                  target = torch.from_numpy(real))
    return mae, rmse, mape2*100

In [151]:
prediction = p['1220025300'].values
real = t['1220025300'].values
prediction2 = pp['1220025300'].values
eval(prediction, real), eval(prediction2, real)

((tensor(3.7574), tensor(4.6843), tensor(19.2001)),
 (tensor(2.4889), tensor(3.1277), tensor(11.7368)))

In [152]:
prediction = p['1220029900'].values
real = t['1220029900'].values
prediction2 = pp['1220029900'].values
eval(prediction, real), eval(prediction2, real)

((tensor(13.0687), tensor(13.2532), tensor(45.2184)),
 (tensor(2.3783), tensor(3.0062), tensor(8.1467)))

In [ ]:
prediction = p['1220025300'].values
real = t['1220025300'].values
prediction2 = pp['1220025300'].values
eval(prediction, real), eval(prediction2, real)

In [ ]:
prediction = p['1220025300'].values
real = t['1220025300'].values
prediction2 = pp['1220025300'].values
eval(prediction, real), eval(prediction2, real)

In [ ]:
prediction = p['1220025300'].values
real = t['1220025300'].values
prediction2 = pp['1220025300'].values
eval(prediction, real), eval(prediction2, real)

In [ ]:
prediction = p['1220025300'].values
real = t['1220025300'].values
prediction2 = pp['1220025300'].values
eval(prediction, real), eval(prediction2, real)

In [ ]:
prediction = p['1220025300'].values
real = t['1220025300'].values
prediction2 = pp['1220025300'].values
eval(prediction, real), eval(prediction2, real)

In [153]:
predDf[['1220029900', '1220027600_2', '1220031100_3']]

,1220029900,1220027600_2,1220031100_3
0,9.380019,13.232187,22.240961
1,9.857939,13.240847,22.079163
2,10.249720,13.511192,22.494946
3,10.358127,14.107889,22.275883
4,10.177292,14.313955,21.918886
...,...,...,...
571,9.927524,13.215752,20.654627
572,9.927524,13.215752,20.654627
573,9.927524,13.215752,20.654627
574,9.927524,13.215752,20.654627


In [ ]:
def evaluation_20_only(y_pred, y_true):
    for i in range(4):
        idx = i * 3 + 2
        prediction = y_pred[idx]
        real = y_true[idx]
        print(idx)
        
        trueDf = pd.DataFrame(real, columns=df.columns)
        predDf = pd.DataFrame(prediction, columns=df.columns)
        
        predDf['1220025300'] = (predDf['1220026300_1'] + predDf['1220025300_3'])/2
        predDf['1220029900'] = (predDf['1220027600_2'] + predDf['1220031100_3'])/2
        predDf['1220026000'] = (predDf['1220024600_0'] + predDf['1220025200_2'])/2
        predDf['1220024700'] = (predDf['1220025800_0'] + predDf['1220027700_1'])/2
        predDf['1220022300'] = (predDf['1220023000_1'] + predDf['1220020600_3'])/2
        predDf['1220026100'] = (predDf['1220029100_1'] + predDf['1220029900_3'])/2
        predDf['1220029000'] = (predDf['1220023100_0'] + predDf['1220025500_2'])/2
        
        for col in predDf.columns:
            if '_' in col:
                predDf = predDf.drop(columns=[col])
                
        for col in trueDf.columns:
            if '_' in col:
                trueDf = trueDf.drop(columns=[col])
                
        columns = ['1220025300', '1220029900', '1220026000', '1220024700',
                  '1220022300', '1220026100', '1220029000']
        
        predDf = predDf[columns]
        trueDf = trueDf[columns]
        
        prediction = predDf.values
        real = trueDf.values

        mae = torchmetrics.functional.mean_absolute_error(
            torch.from_numpy(prediction), 
            torch.from_numpy(real)
        )
        rmse = torch.sqrt(
            torchmetrics.functional.mean_squared_error(
                torch.from_numpy(prediction), 
                torch.from_numpy(real)
            )
        )

        mapes = torchmetrics.MeanAbsolutePercentageError()
        mape2 = mapes(preds = torch.from_numpy(prediction), 
                      target = torch.from_numpy(real))
        mape = np.sum(np.abs(real - prediction) / real) / (real.shape[0] * real.shape[1])

        print((idx+1) * 5, 'Min')
        print('MAE:', mae, 'RMSE:', rmse, 'MAPE:', mape * 100, 'MAPE2:', mape2*100)
        
    return predDf, trueDf